In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip
import speech_recognition as sr
import json

def split_video(video_path, chunk_length):
    with VideoFileClip(video_path) as video:
        duration = int(video.duration)
        for i in range(0, duration, chunk_length):
            start = i
            end = min(i + chunk_length, duration)
            yield video.subclip(start, end), start, end

def extract_and_transcribe(video_clip, start, end, recognizer):
    audio_path = f"chunk_{start:04d}_{end:04d}.wav"
    video_clip.audio.write_audiofile(audio_path)

    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Audio was not understood"
        except sr.RequestError:
            return "Request failed"

def main(video_path):
    recognizer = sr.Recognizer()
    all_transcripts = {}

    for video_clip, start, end in split_video(video_path, 10):
        transcript = extract_and_transcribe(video_clip, start, end, recognizer)
        all_transcripts[f"chunk_{start:04d}_{end:04d}"] = transcript

    with open('full_transcript.json', 'w') as json_file:
        json.dump(all_transcripts, json_file, indent=4)

    print("Transcription completed and saved to full_transcript.json")

# Usage
video_path = "Lunch.mp4" 
main(video_path)